In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import os
os.chdir('/home/stud/ath/ath_ws/keypoint_dataset_pipeline')

In [53]:
from config import config
from utils import logger

import numpy as np
import h5py
import random
import torch

In [54]:
def print_hdf5_structure(reader):
    def print_group(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
            
    reader._file.visititems(print_group)

In [55]:
class StatsReader:
    def __init__(self, train=False):
        filepath = f'{config.paths[config.task.name].output}/data.hdf5'
        self._file = h5py.File(filepath, 'r')
        
        self._init_groups_read_mode()

    def _init_groups_read_mode(self):
        self._detector = self._file[f'{config.task.cam}/detector']
        self._matcher = self._file[f'{config.task.cam}/matcher']
        self._filter = self._file[f'{config.task.cam}/filter']
        self._matches = self._file[f'{config.task.cam}/matches']

        self.detector_normalised = self._detector['normalised']
        self.detector_confidences = self._detector['confidences']

        self.matcher_warp = self._matcher['warp'] # only one you will need
        self.matcher_certainty = self._matcher['certainty']

        self.filter_normalised = self._filter['normalised']
        self.filter_confidences = self._filter['confidences']

        self.cropped_image_reference_coords = self._matches['crop/reference_coords']
        self.cropped_image_target_coords = self._matches['crop/target_coords']
    
    def close(self):
        self._file.close()


In [56]:
config.task.frame_filtering = 'normal'

reference_coords_total_count = 0
target_coords_total_count = 0

coords_stats = {}

for track in config.task.tracks:
    config.task.track = track
    logger.info(f'Track : {track}')
    config.task.dataset_kind = track[:2]

    filepath = f'{config.paths[config.task.name].output}/data.hdf5'
    print(filepath)

    reader = StatsReader()
    # print_hdf5_structure(reader)

    coords_stats[track] = 0

    for cam in config.task.cams:
        config.task.cam = cam
        logger.info(f'Cam : {cam}')

        refs_from = reader._file[f'{cam}/matches/crop/reference_coords']
        tars_from = reader._file[f'{cam}/matches/crop/target_coords']

        print(f'Number of Reference Coords  : {len(refs_from)}')
        print(f'Number of Target Coords     : {len(tars_from)}')

        coords_stats[track] += len(refs_from)

        reference_coords_total_count += len(refs_from)
        target_coords_total_count += len(tars_from)
    
    reader.close()
    
print(f'Total Reference Coords : {reference_coords_total_count}')
print(f'Total Target Coords : {target_coords_total_count}')

INFO     | <module> | Track : MOO01_hand_puncher_1
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO01_hand_puncher_1/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 610
Number of Target Coords     : 610
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 610
Number of Target Coords     : 610
INFO     | <module> | Track : MOO02_hand_puncher_2
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO02_hand_puncher_2/data.hdf5
INFO     | <module> | Cam : cam0


Number of Reference Coords  : 959
Number of Target Coords     : 959
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 961
Number of Target Coords     : 961
INFO     | <module> | Track : MOO03_hand_shooter_easy
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO03_hand_shooter_easy/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 486
Number of Target Coords     : 486
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 489
Number of Target Coords     : 489
INFO     | <module> | Track : MOO04_hand_shooter_hard
/home/stud/ath/ath_ws/keypoint_dataset_pipeline/output_normal/basalt/monado_slam/MOO04_hand_shooter_hard/data.hdf5
INFO     | <module> | Cam : cam0
Number of Reference Coords  : 832
Number of Target Coords     : 832
INFO     | <module> | Cam : cam1
Number of Reference Coords  : 825
Number of Target Coords     : 825
INFO     | <module> | Track : MOO05_inspect_easy
/home/stud/ath/ath_ws/keypoint_dataset_p

# Train Data

In [60]:
filepath = f'{config.paths[config.task.name].train_data}/train_data.hdf5'
train_data_f = h5py.File(filepath, 'r')

In [61]:
train_data_f

<HDF5 file "train_data.hdf5" (mode r)>

In [70]:
total_pairs = 0
coords_stats = {}

def print_hdf5_structure(train_data_f):
    def print_group(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name} = {len(obj)}")
            track = name.split('/')[0]

            if track not in coords_stats.keys():
                coords_stats[track] = 0

            if name.endswith('reference_coords'):
                global total_pairs
                total_pairs += len(obj)
                logger.info(f'Length so far {total_pairs}')

                coords_stats[track] += len(obj) 
            
    train_data_f.visititems(print_group)

print_hdf5_structure(train_data_f)
logger.info(f'Length {total_pairs}')

Group: MGO01_low_light = 2
Group: MGO01_low_light/cam0 = 4
Group: MGO01_low_light/cam0/indices = 535
Group: MGO01_low_light/cam0/reference_coords = 535
INFO     | print_group | Length so far 535


Group: MGO01_low_light/cam0/rotations = 535
Group: MGO01_low_light/cam0/target_coords = 535
Group: MGO01_low_light/cam1 = 4
Group: MGO01_low_light/cam1/indices = 562
Group: MGO01_low_light/cam1/reference_coords = 562
INFO     | print_group | Length so far 1097
Group: MGO01_low_light/cam1/rotations = 562
Group: MGO01_low_light/cam1/target_coords = 562
Group: MGO02_hand_puncher = 2
Group: MGO02_hand_puncher/cam0 = 4
Group: MGO02_hand_puncher/cam0/indices = 823
Group: MGO02_hand_puncher/cam0/reference_coords = 823
INFO     | print_group | Length so far 1920
Group: MGO02_hand_puncher/cam0/rotations = 823
Group: MGO02_hand_puncher/cam0/target_coords = 823
Group: MGO02_hand_puncher/cam1 = 4
Group: MGO02_hand_puncher/cam1/indices = 801
Group: MGO02_hand_puncher/cam1/reference_coords = 801
INFO     | print_group | Length so far 2721
Group: MGO02_hand_puncher/cam1/rotations = 801
Group: MGO02_hand_puncher/cam1/target_coords = 801
Group: MGO03_hand_shooter_easy = 2
Group: MGO03_hand_shooter_easy

# Order videos

In [72]:
sorted_items = sorted(coords_stats.items(), key=lambda item: item[1])

for key, value in sorted_items:
    print(f"{key} : {value}")

MOO09_short_1_updown : 12
MGO09_short_1_updown : 15
MGO10_short_2_panorama : 42
MOO10_short_2_panorama : 42
MGO11_short_3_backandforth : 82
MOO11_short_3_backandforth : 82
MOO07_mapping_easy : 178
MOO08_mapping_hard : 203
MGO08_mapping_hard : 221
MGO07_mapping_easy : 293
MOO05_inspect_easy : 371
MGO05_inspect_easy : 530
MGO14_flickering_light : 810
MOO03_hand_shooter_easy : 950
MGO03_hand_shooter_easy : 1055
MGO01_low_light : 1097
MOO06_inspect_hard : 1111
MOO01_hand_puncher_1 : 1218
MGO06_inspect_hard : 1279
MOO14_flickering_light : 1422
MGO04_hand_shooter_hard : 1474
MGO13_sudden_movements : 1491
MOO04_hand_shooter_hard : 1563
MGO02_hand_puncher : 1624
MOO02_hand_puncher_2 : 1844
MOO13_sudden_movements : 2093


In [83]:
# Calculate the total sum of values
total_sum = sum(coords_stats.values())
validation_target = total_sum * 0.25

# Split into validation and training based on value sum
validation = {}
training = {}
current_sum = 0

for key, value in sorted_items:
    if current_sum + value <= validation_target:
        validation[key] = value
        current_sum += value
    else:
        training[key] = value

validation_sum = sum(validation.values())
training_sum = sum(training.values())

print("Validation Set:")
print(f"Count of values (sum): {validation_sum}\n")
for key, value in validation.items():
    print(f"{key}: {value}")

print("\nTraining Set:")
print(f"Count of values (sum): {training_sum}\n")
for key, value in training.items():
    print(f"{key}: {value}")

Validation Set:
Count of values (sum): 4886

MOO09_short_1_updown: 12
MGO09_short_1_updown: 15
MGO10_short_2_panorama: 42
MOO10_short_2_panorama: 42
MGO11_short_3_backandforth: 82
MOO11_short_3_backandforth: 82
MOO07_mapping_easy: 178
MOO08_mapping_hard: 203
MGO08_mapping_hard: 221
MGO07_mapping_easy: 293
MOO05_inspect_easy: 371
MGO05_inspect_easy: 530
MGO14_flickering_light: 810
MOO03_hand_shooter_easy: 950
MGO03_hand_shooter_easy: 1055

Training Set:
Count of values (sum): 16216

MGO01_low_light: 1097
MOO06_inspect_hard: 1111
MOO01_hand_puncher_1: 1218
MGO06_inspect_hard: 1279
MOO14_flickering_light: 1422
MGO04_hand_shooter_hard: 1474
MGO13_sudden_movements: 1491
MOO04_hand_shooter_hard: 1563
MGO02_hand_puncher: 1624
MOO02_hand_puncher_2: 1844
MOO13_sudden_movements: 2093
